### Ensemble Learning

Ensemble Learning은 여러 개의 동질적이거나 base model 혹은 weak learner를 동일한 문제에 대해 다른 표본으로 훈련시킨 후 결과를 종합하거나 순차적으로 훈련시켜서 예측의 분산과 편향을 줄여준다

종류 : Bagging (Random Forest), Boosting(Ada Boost, GBM계열)

Ada Boost 계열 : SAMME

GBM 계열 : XG, LGBM, Catboost

Random Forest Hyperparameter 소개

#### 1. Bagging

배깅(Bagging)은 Ensemble Learning의 하나로, sklearn에서는 `BaggingClassifier`와 `RandomForestClassifier`가 존재한다 (회귀 모형 또한 존재한다)


**1.1 Booststrap Sampling**

Bagging(Bootstrap Aggregation)은 예측 시 분산을 축소하는 효과적인 방법이다. 배깅은 다음과 같은 단계를 따른다. 첫째, Bootstrap Sampling을 통해 N개의 Training Data set을 생성한다. 주어진 data set을 모집단으로 가정하고, 여러개의 sub dataset을 추출하여 표본집단으로 가정한 뒤 이 과정을 여러번 복원추출하여 사용한다.

$$\mathcal{D}\left(x_i,y_i\right)=\{\left(x_1,y_1\right),\left(x_2,y_2\right),\ldots,\left(x_n,y_n\right)\}$$
$$\mathcal{D}\left(x_i^\ast,y_i^\ast\right)=\{\left(x_1^\ast,y_1^\ast\right),\left(x_2^\ast,y_2^\ast\right),\ldots,\left(x_n^\ast,y_n^\ast\right)\}$$

둘째, 각각의 Training Set에 적합화된 N개의 추정 계수를 구한다. 이 추정 계수들을 각각 독립적이로 추정되기 때문에 모형은 병렬로 적합화될 수 있다

셋째, Ensemble Learner의 예측을 N개의 모델로부터 생성된 각 개별 예측의 단순 평균이다. Categorical 변수의 경우 관측치가 어떤 클래스에 속할 확률은 관측을 그 클래스 멤버로 분류한 추정기의 비율로 계산한다. 기초 추정기가 예측 확률을 갖고 예측할 수 있으면 배깅 분류기는 확률의 평균을 도출할 수 있다.

**1.2 분산 축소 알고리즘**

배깅의 주요 장점은 예측의 분산을 줄이는 것이다. 그러므로, Overfitting을 해결하는데 도움이 된다. 배깅 모형의 예측 분산은 배깅의 추정 계수 (N), 단일 추정 계수 예측의 평균, 분산, 그리고 이들 예측 간의 평균 상관관계에 대한 함수이다

$$
\begin{matrix}
V\left[ \frac{1}{N} \sum_{i=1}^N \varphi_i[c]\right]  & = & \frac{1}{N^2}\sum_{i=1}^N \left( \sum_{j=1}^N \sigma_{i,j}\right) \\
& = & \frac{1}{N^2} \sum_{i=1}^N \left( \sigma_i^2 + \sum_{j \neq i}^N \sigma_i \sigma_j \rho_{i,j}\right) \\
& = & \frac{1}{N^2}\sum_{i=1}^N \left( \bar{\sigma}^2 + \sum_{j \neq i}^N \bar{\sigma}^2 \bar{\rho}\right) \\
& = & \frac{\bar{sigma}^2 + (N-1)\bar{\sigma}^2 \bar{\rho}}{N} \\
& = & \bar{\sigma}^2 \big( \bar{\rho} + \frac{1 - \bar{\rho}}{N}\big)
\end{matrix}
$$

여기서 $\sigma_{i,j}$는 추정기 $i,j$에 의한 predicted covariance이다

$$\sum_{i=1}^{N}\bar{\sigma^2}=\sum_{i=1}^{N}\sigma_i^2\Leftrightarrow\bar{\sigma^2}N^{-1}\sum_{i=1}^{N}\sigma_i^2$$
$$\sum_{j\neq i}^{N}{\bar{\sigma_i^2}\bar{\rho}}=\sum_{j\neq i}^{N}{\sigma_i\sigma_j\sigma_{i,j}}\Leftrightarrow\bar{\rho^2}=\left(\sigma^2N\left(N-1\right)\right)^{-1}\sum_{j\neq i}^{N}{\sigma_i\sigma_j\sigma_{i,j}}$$

위 식에서 $\bar{\rho}\rightarrow1\Rightarrow V\left[\frac{1}{2}\sum_{i=1}^{N}{\varphi_i\left[c\right]}\right]\rightarrow\bar{\sigma^2}$이므로 배깅이 $\bar{\rho}<1$에 대해서만 효과적이라는 것을 식에서 보여 준다. 즉, Machine Learning Algorithm이라고 어떠한 데이터를 넣어도 되는 것이 아니라 추정 성능의 극대화를 위해서는 데이터의 품질이 매우 중요하다

**1.3 개선된 정확도**

$N$개의 독립된 분류 기간의 다수결에 의해 $k$개 클래스를 예측하는 배깅 분류기를 생각해 보자. 예측은 ${0,1}$로 labeling할 수 있는데, 여기서 1은 정확한 예측을 의미한다. 분류기의 정확도는 예측을 1로 labeling하는 확률 p이다. 평균적으로 $Np$예측이 1로 labeling될 것이고, 분산은 $Np\left(1-p\right)$이다. 다수결 투표는 가장 많이 예측된 클래스가 관측된 경우에 정확한 예측을 한 것이다

예를 들어, $N\ =\ 10$이고 $k=3$일때 배깅 분류기는 클래스 A가 관측되고, 투표의 결과가 $\left[A,B,C\right]=\left[4,3,3\right]$이었다면 정확한 예측을 한 것이다. 그러나 배깅 분류기는 클래스 A가 관측되고, 투표의 결과가 $\left[A,B,C\right]=\left[4,1,5\right]$라면 예측에 실패를 한 것이다. 충분 조건은 이 label의 합이 $X>\frac{X}{2}$인 것이다. 필요조건은 $X>\frac{X}{k}$이고, 이는 다음과 같은 확률로 발생한다

$$P\left[X>\frac{N}{k}\right]=1-P\left[X\le\frac{N}{k}\right]=1-\sum_{i=0}^{\left\lfloor\frac{N}{k}\right\rfloor}{\binom{N}{i}p^i\left(1-p\right)^{N-i}}$$

이는 N이 충분히 크면, 즉 $N>p\left(p-\frac{1}{k}\right)^{-2}$이면 $p>1\Rightarrow P\left[X>\frac{N}{k}\right]>p$이라는 것을 의미한다. 따라서 배깅 분류기의 정확도는 **개별 분류기들의 평균 정확도를 능가**한다

**1.4** Random Forest

<center>
<img src="https://miro.medium.com/v2/resize:fit:1400/1*jE1Cb1Dc_p9WEOPMkC95WQ.png" title="RF"/>
</center>

Decision Tree는 과적합되기 쉽다고 알려져 있으며, 이는 예측의 분산을 증가시킨다. Random Forest는 이 문제를 해결하기 위해서 더 낮은 분산을 가진 Ensemble Learning을 하도록 설계되었다

Random Forest는 데이터 중 Bootstrapping된 부분 집합에 대해 Decision Tree 가 추정한 parameter를 독립적으로 학습해 산출한다는 점에서 배깅 알고리즘과 비슷하다. 배깅과 비교해 핵심적인 차이는 Random Forest가 2차 수준의 무작위성을 반영한다는 점이다. 각 node의 분기를 최적화할 떄 속성의 Random subsample만 평가하여 추정기간 상관관계를 더욱 축소하려는 목적을 가지고 있다

Random Forest가 가진 장점으로는 다음과 같다

1. Random Forest가 Bagging처럼 예측 분산을 과적합 없이 줄일수 있다는 것이다 (단, $\bar{\rho} < 1$의 조건이 충족되어야 한다)
2. Random Forest가 Feature Importance를 계산할 수 있는데, 이에 대해서는 마지막 주차에 심층적으로 다룰 예정이다
3. Random Forest가 Out of bag의 정확도를 계산할 수 있다는 것이다. 그러나, 금융 데이터에 적용 시에는 부풀려질 가능성이 존재한다

Bagging Algorithm에서처럼 Random Forest도 개별 Decision Tree보다 더 낮은 편향을 보이지는 않을 것이다

#### 2. Boosting

**2.1 AdaBoost**

**2.2 Gradient Boosting**

**2.3 Cat Boost**

#### 3. Boosting vs Bagging